https://chrisalbon.com/machine_learning/trees_and_forests/random_forest_classifier_example/


In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

iris = datasets.load_iris()
X = iris.data[:,2:]  #:2 we only take the first two features.
y = iris.target

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())

plt.show()

<Figure size 800x600 with 1 Axes>

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2138)

In [1]:
from Dotua.rautodiff import rAutoDiff as rad
from Dotua.roperator import rOperator as op
import random

ad = rad()

class NeuralNetwork():
	def __init__(self, input_vals, input_bias, hidden_bias, num_hidden, output, learning_rate = 0.1):
		self.input_vals = input_vals
		self.input_bias = input_bias
		self.hidden_bias = hidden_bias
		self.num_hidden = num_hidden
		self.output = output
		self.learning_rate = learning_rate

		# To initialize the weights from input layer to hidden layer
		self.weights_tohidden = [None] * num_hidden
		for i in range(num_hidden):
			self.weights_tohidden[i] = []
			for j in range(len(input_vals)):
				w = ad.create_rscalar(random.random())
				self.weights_tohidden[i].append(w)

		# To initialize the weights from hidden layer to output layer
		self.weights_tooutput = [None] * len(output)
		for i in range(len(output)):
			self.weights_tooutput[i] = []
			w = ad.create_rscalar(random.random())
			for j in range(num_hidden):
				self.weights_tooutput[i].append(w)

	def train(self, input_vals, output_vals):
		self.input_vals = input_vals
		self.output = output_vals

		# To calculate the hidden layer neurons
		self.hidden_layer = []
		for i in range(self.num_hidden):
			h = 0
			for j in range(len(self.weights_tohidden[i])):
				h = h + self.weights_tohidden[i][j] * self.input_vals[j]
			h = h + self.input_bias
			self.hidden_layer.append(1/(1+op.exp(-h)))

		# To calculate the output layer neurons and error
		error = 0
		for i in range(len(self.output)):
			o = 0
			for j in range(len(self.weights_tooutput[i])):
				o = o + self.weights_tooutput[i][j] * self.hidden_layer[j]
			o = o + self.hidden_bias
			o = 1/(1+op.exp(-o))
			error = error + (o - self.output[i]) ** 2

		# To update weights from hidden layer to output layer
		for i in range(len(self.weights_tooutput)):
			for j in range(len(self.weights_tooutput[i])):
				d = ad.partial(error, self.weights_tooutput[i][j])
				self.weights_tooutput[i][j] = self.weights_tooutput[i][j] - d * self.learning_rate

		# To update weights from input layer to hidden layer
		for i in range(len(self.weights_tohidden)):
			for j in range(len(self.weights_tooutput[i])):
				d = ad.partial(error, self.weights_tohidden[i][j])
				self.weights_tohidden[i][j] = self.weights_tohidden[i][j] - d * self.learning_rate

	def predict(self, input_vals, output_vals):
		self.input_vals = input_vals
		self.output = output_vals

		# To calculate the hidden layer neurons using the current model
		self.hidden_layer = []
		for i in range(self.num_hidden):
			h = 0
			for j in range(len(self.weights_tohidden[i])):
				h = h + self.weights_tohidden[i][j] * self.input_vals[j]
			h = h + self.input_bias
			self.hidden_layer.append(1/(1+op.exp(-h)))

		# To calculate the output layer neurons using the current model and calculate the error
		error = 0
		output_layer = []
		for i in range(len(self.output)):
			o = 0
			for j in range(len(self.weights_tooutput[i])):
				o = o + self.weights_tooutput[i][j] * self.hidden_layer[j]
			o = o + self.hidden_bias
			o = 1/(1+op.exp(-o))
			output_layer.append(o.val)
			error = error + (o - self.output[i]) ** 2
		error = error / len(self.output)
		return (output_layer, error.val)

In [2]:
nn = NeuralNetwork([0.05,0.1],0.35,0.6,2,[0.01,0.09])
for i in range(100):
	nn.train([0.05,0.1],[0.01,0.09])
output, e = nn.predict([0.05,0.1],[0.01,0.09])
print('Final prediction given by the Neural Network is ', output)
print('The mean squared error is ', e)

Final prediction given by the Neural Network is  [0.36576268400301354, 0.35000176859821175]
The mean squared error is  0.09708400350161306


In [9]:
import numpy as np
def get_y(i):
    train_y = [0,0,0]
    train_y[i] = 1
    return train_y

nn = NeuralNetwork(X_train[0],0.35,0.6,2,get_y(y_train[0]))

prediction = []
for j in range(10):
    for i in range(1, len(X_train)):
        nn.train(X_train[i], get_y(y_train[i]))
    
for i in range(1, len(X_test)):
    output,e = nn.predict(X_test[i], get_y(y_test[i]))
    print('Final prediction given by the Neural Network is ', output)
    print('The mean squared error is ', e)

Final prediction given by the Neural Network is  [0.3355135962300062, 0.31790696161716203, 0.34875646122104836]
The mean squared error is  0.2331504518363173
Final prediction given by the Neural Network is  [0.33237288245745084, 0.3172624658169574, 0.3495010260506076]
The mean squared error is  0.23291774692861442
Final prediction given by the Neural Network is  [0.3454659609627771, 0.3201732068326598, 0.3467701024583335]
The mean squared error is  0.2339202342836959
Final prediction given by the Neural Network is  [0.4136130230361116, 0.35029127503814894, 0.3549618248654667]
The mean squared error is  0.1975171870775074
Final prediction given by the Neural Network is  [0.3171780149707294, 0.3138961025793877, 0.3528503959110166]
The mean squared error is  0.20597842215594328
Final prediction given by the Neural Network is  [0.3259601199824146, 0.31585584193501204, 0.35091247167728873]
The mean squared error is  0.2324809305373551
Final prediction given by the Neural Network is  [0.3154

In [53]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential

# Save the number of columns in predictors: n_cols
n_cols = X.shape[1]

# Set up the model: model
model = Sequential()

# Add the first layer
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))

# Add the second layer
model.add(Dense(32, activation='relu'))

# Add the output layer
model.add(Dense(1))


In [54]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential

# Specify the model
n_cols = X.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Verify that model contains information from compiling
print("Loss function: " + model.loss)

Loss function: mean_squared_error


In [57]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential

# Specify the model
n_cols = X.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(X, y)


Epoch 1/1
150/150 [==============================] - 0s 2ms/step - loss: 0.9938


In [ ]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical

# Convert the target to categorical: target
target = to_categorical(df.survived)

# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(32, activation='relu', input_shape=(n_cols,)))

# Add the output layer
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Fit the model
model.fit(predictors, target)
